In [1]:
using Interpolations, Plots, Unitful, StructArrays, UnPack, HDF5
using Flight.Atmosphere: ISA_layers, AirProperties, p_std, T_std, g_std, R
using Flight.Geodesy: HGeop

p_std_inHg = 101325 / 3386.389

ω_rated = 2700 #RPM
P_rated = 200 #HP

n_idle = 0.25
n_max = 1.2

π_idle_std = 0.01
μ_idle_std = 9/p_std_inHg
π_n_max = π_idle_std

fname = "piston.h5"

ERROR: 

LoadError: LoadError: 

unable to determine if src/ww15mgh_hdf5.h5 is accessible in the HDF5 format (file may not exist)
Stacktrace:
  [1] 

error(s::String)
    @ 

Base .\error.jl:33
  [2] h5open(filename::String, mode::String, fapl::HDF5.FileAccessProperties, fcpl::HDF5.FileCreateProperties; swmr::Bool)
    @ HDF5 ~\.julia\packages\HDF5\t07BK\src\HDF5.jl:248
  [3] h5open(filename::String, mode::String; swmr::Bool, pv::Base.Iterators.Pairs{Union{}, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
    @ HDF5 ~\.julia\packages\HDF5\t07BK\src\HDF5.jl:266
  [4] h5open(f::Flight.Geodesy.var"#7#8"{Matrix{Float32}}, args::String; swmr::Bool, pv::Base.Iterators.Pairs{Union{}, Union{}, Tuple{}, NamedTuple{(), Tuple{}}})
    @ HDF5 ~\.julia\packages\HDF5\t07BK\src\HDF5.jl:285
  [5] h5open
    @ ~\.julia\packages\HDF5\t07BK\src\HDF5.jl:285 [inlined]
  [6] load_geoid_data_hdf5(file_path::String)
    @ Flight.Geodesy ~\Repos\Julia\Flight\src\geodesy.jl:187
  [7] load_geoid_data_hdf5()
    @ Flight.Geodesy ~\Repos\Julia\Flight\src\geodesy.jl:186
  [8] top-level scope
    @ ~\Repos\Julia\Flight\src\geodesy.jl:195
  [9] include(mod::Module, _path::String)
    @ Base 

ErrorException: Failed to precompile Flight [e329eb8f-69f3-4234-a468-5e3889a58643] to C:\Users\Miguel\.julia\compiled\v1.6\Flight\jl_EDB3.tmp.

### Definitions

For implementation, define the following normalized variables
$$\pi = \dfrac{P}{P_{rated}}$$
$$n = \dfrac{\omega}{\omega_{rated}}$$
$$\mu = \dfrac{MAP}{p_{std}}$$

In [ ]:
β = ISA_layers[1].β

δ_from_p(p) = (p/p_std) ^ (1+β*R/(2*g_std)) #computes normalized δ 

function δ_from_h(h) #computes normalized δ
    @unpack p, T = AirProperties(HGeop(h))
    p / p_std / √(T / T_std)
end

@assert δ_from_h(1345) ≈δ_from_p(AirProperties(HGeop(1345)).p)

### Part Throttle Power at Standard Conditions

We need a 2D table providing $\pi_{ISA,std}(n,\mu)$.

From the graph we see that the function $\pi_{ISA,std}(n,\mu)$ is linear in $\mu$. Therefore we need only two data points for each $n$: $(\mu_{1}(n), \pi_{ISA,std}(n,\mu_1(n)))$ and $(\mu_{2}(n), \pi_{ISA,std}(n, \mu_2(n)))$.

However, for a 2D table we need these two $\mu$ values to be the same across all values of $n$. So for each $n$, we construct a 1D linear interpolator, resample $\pi_{std}$ at two $\mu$ values, arbitrary but equal for all $n$, then build the 2D interpolator.

For $n > 1$, if we were to simply extrapolate the values in the graph, power would increase indefinitely with engine speed, which we know is not realistic. To avoid this, we add two extra $\pi$ entries at $n_{N+2}, n_{N+1} > 1$, with power reaching a peak between $n_{N}$ and $n_{N+1}$, then decreasing between $n_{N+1}$ and $n_{N+2}$, both for $\mu_1$ and $\mu_2$. The values of $\mu_1$ and $\mu_2$ for these synthetic $n$ values are simply extrapolations. 


In [ ]:
function build_interp_π_ISA_std()

    n_range_π = [n_idle*ω_rated, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2900, n_max*ω_rated]/ω_rated

    μ_std_1 = [μ_idle_std*p_std_inHg, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, μ_idle_std*p_std_inHg]/p_std_inHg #each value in n_range_π
    π_ISA_std_1 = [π_idle_std*P_rated 54 61 67 72 76 81 85.5 90 95.1 99.5 99.5 π_idle_std*P_rated] / P_rated  |> vec

    μ_std_2 = [p_std_inHg 25 25.55 26.15 26.88 27.3 28.1 28.75 28.7 28.65 28.6 28.50 p_std_inHg] / p_std_inHg |> vec
    π_ISA_std_2 = [1.8π_idle_std*P_rated 97.8 109.5 121.8 136 145.75 162 176 184 193 200 200 π_idle_std*P_rated] / P_rated |> vec
    
    μ_range_π = [μ_idle_std*p_std_inHg, 17, 30] / p_std_inHg

    π_ISA_std_data = Array{Float64,2}(undef, (length(n_range_π), length(μ_range_π)))
    for (i, n) in enumerate(n_range_π)
        interp_π_ISA_std_1D = LinearInterpolation([μ_std_1[i], μ_std_2[i]], [π_ISA_std_1[i], π_ISA_std_2[i]], extrapolation_bc = Line())
        π_ISA_std_data[i,:] = interp_π_ISA_std_1D.(μ_range_π)
    end

    interp_π_ISA_std = LinearInterpolation((n_range_π, μ_range_π), π_ISA_std_data, extrapolation_bc = ((Flat(), Flat()), (Flat(), Line())))
end

interp_π_ISA_std = build_interp_π_ISA_std()

@show interp_π_ISA_std(0.25, 0.25) * P_rated
@show interp_π_ISA_std(1800/ω_rated, 17/p_std_inHg) * P_rated
@show interp_π_ISA_std(2400/ω_rated, 24/p_std_inHg) * P_rated
@show interp_π_ISA_std(2500/ω_rated, 26.5/p_std_inHg) * P_rated
@show interp_π_ISA_std(1950/ω_rated, 20.5/p_std_inHg) * P_rated
@show interp_π_ISA_std(2150/ω_rated, 27/p_std_inHg) * P_rated

# n_plot = [0.25ω_rated, 0.35ω_rated, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 1.2ω_rated]/ω_rated
n_plot = range(0.15, 1.5, length = 100)
δ_plot = range(1, δ_from_h(32000*0.3048), length = 10)
μ_plot = range(0.25p_std_inHg, 30, length = 10)/p_std_inHg

π_std_plot = [interp_π_ISA_std(n, μ) for (n, μ) in Iterators.product(n_plot, μ_plot)]
# plot(μ_plot, π_std_plot')
plot(n_plot, π_std_plot)

# h5open(fname, "w") do fid

#     create_group(fid, "pi_ISA_std")
#     π_ISA_std = fid["pi_ISA_std"]

#     π_ISA_std["n"] = collect(n_range)
#     π_ISA_std["mu"] = collect(μ_range_std)
#     π_ISA_std["data"] = π_ISA_std_data

# end


### $\delta_{wot}(n, \mu)$

In [ ]:
# function build_δ_wot_interp()

#     n_1_wot = 1800/ω_rated
#     n_2_wot = 2700/ω_rated

#     #take the pressure altitude values in kft read from the graph and convert them
#     #to meters
#     h_wot_1 = ustrip.(u"m", [22.2, 18.55, 15.4, 12.5, 9.8, 7.35, 5.1, 3.00, 1.1]*1000u"ft")
#     h_wot_2 = ustrip.(u"m", [21.7, 18.2, 15.0, 12.1, 9.45, 6.95, 4.7, 2.5, 0.6]*1000u"ft")

#     #compute the corresponding δ values
#     @show δ_wot_1 = δ_from_h.(h_wot_1)
#     @show δ_wot_2 = δ_from_h.(h_wot_2)

#     #for each normalized manifold pressure, construct a δ_wot(n) interpolation

#     n_range_wot = range(1800, 2700, step = 100)/ω_rated
#     μ_range_wot = range(12, 28, step = 2)/p_std_inHg #24 26 28
#     δ_wot_data = Array{Float64}(undef, length(n_range_wot), length(μ_range_wot))
#     for (i, μ) in enumerate(μ_range_wot)
#         interp_δ_wot_1D = LinearInterpolation([n_1_wot, n_2_wot], [δ_wot_1[i], δ_wot_2[i]], extrapolation_bc = Line())
#         δ_wot_data[:,i] = interp_δ_wot_1D.(n_range_wot)
#     end

#     # interp_δ_wot = extrapolate(scale(interpolate(δ_wot_data, BSpline(Linear())), n_range_wot, μ_range_wot), Line())
#     interp_δ_wot = LinearInterpolation((n_range_wot, μ_range_wot), δ_wot_data, extrapolation_bc = Line())
# end

function build_δ_wot_interp()

    n_range = [1800/ω_rated, 2700/ω_rated]
    @show μ_range = range(12, 28, step = 2)/p_std_inHg #24 26 28

    #take the pressure altitude values in kft read from the graph and convert them
    #to meters
    @show h_data = ustrip.(u"m", [22.2 18.55 15.4 12.5 9.80 7.35 5.1 3.00 1.1
                            21.7 18.20 15.0 12.1 9.45 6.95 4.7 2.50 0.6]*1000u"ft")

    @show δ_data = δ_from_h.(h_data)

    # interp_δ_wot = extrapolate(scale(interpolate(δ_data, BSpline(Linear())), n_range, μ_range), Line())
    interp_δ_wot = LinearInterpolation((n_range, μ_range), δ_data, extrapolation_bc = Line())
end
interp_δ_wot = build_δ_wot_interp()

@show interp_δ_wot(2400/ω_rated, 24/p_std_inHg), δ_from_h(4850*0.3048)
@show interp_δ_wot(2100/ω_rated, 20/p_std_inHg), δ_from_h(9600*0.3048)
@show interp_δ_wot(1800/ω_rated, 28/p_std_inHg), δ_from_h(1200*0.3048)
@show interp_δ_wot(1, 28.65/p_std_inHg), δ_from_h(0*0.3048)

δ_wot_plot = [interp_δ_wot(n,μ) for (n,μ) in Iterators.product(n_plot, μ_plot)]
plot(μ_plot, δ_wot_plot')
# plot(n_plot, δ_wot_plot)

### $\mu_{wot}(n, \delta)$

For each $n$ obtain a 1D linear interpolator $\mu_{wot,1D}(\delta)$. Then evaluate it at some predefined $\delta$ range. Finally, use the results for all $n$ values to construct the 2D interpolator $\mu_{wot}(n, \delta)$.

In [ ]:
function build_interp_μ_dot()
    @show δ_range_wot = range(δ_from_h(23e3*.3048), 1, length = 100)

    @show n_range_wot = range(1800, 2700, step = 100)/ω_rated
    @show μ_range_wot = range(12, 28, step = 2)/p_std_inHg #24 26 28
    @show μ_wot_data = Array{Float64}(undef, length(n_range_wot), length(δ_range_wot))
    for (i, n) in enumerate(n_range_wot)
        @show δ_wot_1D = interp_δ_wot(n, μ_range_wot)
        @show interp_μ_wot_1D = LinearInterpolation(δ_wot_1D, μ_range_wot, extrapolation_bc = Line())
        @show μ_wot_data[i, :] = interp_μ_wot_1D.(δ_range_wot)
    end

    # interp_μ_wot = extrapolate(scale(interpolate(μ_wot_data, BSpline(Linear())), n_range_wot, δ_range_wot), Line())
    interp_μ_wot = LinearInterpolation((n_range_wot,δ_range_wot),μ_wot_data)

end

interp_μ_wot =build_interp_μ_dot()

@show interp_μ_wot(1, interp_δ_wot(1, 1))
@show interp_μ_wot(1800/ω_rated, δ_from_h(12500*0.3048)) * p_std_inHg
@show interp_μ_wot(2050/ω_rated, δ_from_h(5000*0.3048)) * p_std_inHg

μ_wot_plot = [interp_μ_wot(n,p) for (n,p) in Iterators.product(n_plot, δ_plot)]
plot(δ_plot, μ_wot_plot')

#μ(n, thr, δ) = μ_wot(n, δ)(μ_idle_ratio + thr * (1-μ_idle_ratio))

#for n_idle = 0.25, thr = 0, δ = 1 (std)
#μ(n_idle, 0, 1) = μ_wot(n_idle, 1) * μ_idle_ratio.

#therefore, by specifying μ(n_idle, 0, 1) = μ_std_idle we can compute
#idle_ratio as:
# @show interp_μ_wot(0.25, 1)
# @show interp_μ_wot(1, 1)
# @show μ_idle_ratio = 0.25 / interp_μ_wot(0.25, 1)

#here
#μ_std_idle = 0.25
#π_std_idle = 0.01

#note that, at altitude and part throttle, μ may be even smaller than
#μ_std_idle. we need to be aware of this when constructing the WOT tables
    
# h5open(fname, "w") do fid

#     create_group(fid, "mu_wot")
#     μ_wot = fid["mu_wot"]

#     μ_wot["n"] = collect(n_range)
#     μ_wot["delta"] = collect(δ_range_wot)
#     μ_wot["data"] =μ_wot_data

# end

Is it justified for the $\mu_{wot}(n,\delta)$ and $\delta_{wot}(n,\mu)$ mappings to be extrapolated linearly? We can reason this with $\mu_{wot}(n,\delta)$; since one mapping is the inverse of the other, and they are linear, the conclusion will apply for both of them. For a given $n$, as $\delta$ grows, $\mu_{wot}$ will keep growing indefinitely, since the air pressure keeps increasing. And for a given $\delta$, as $n$ grows, $\mu_{wot}$ will also keep falling indefinitely, since the engine sucks more air and the manifold vacuum increases. The fact that the brake power decreases due to increasing losses for $n > 1$ does not change these facts.


In [9]:
function build_interp_π_wot()

    @show n_range_π = [n_idle*ω_rated, 1800, 2700, 2900, n_max*ω_rated]/ω_rated

    @show δ_wot_1 = δ_from_h.([40, 23, 23, 23, 40]*1000*.3048)
    @show π_ISA_wot_1 = [π_idle_std*P_rated 46 81.8 81.8 π_idle_std*P_rated] / P_rated |> vec

    δ_wot_2 = ones(length(n_range_π))
    @show π_ISA_wot_2 = [interp_π_ISA_std(n, interp_μ_wot(n, 1)) for n in n_range_π]

    @show δ_range_π = δ_from_h.([40, 0]*1000*0.3048)

    π_ISA_wot_data = Array{Float64,2}(undef, (length(n_range_π), length(δ_range_π)))
    for (i, n) in enumerate(n_range_π)
        interp_π_ISA_wot_1D = LinearInterpolation([δ_wot_1[i], δ_wot_2[i]], [π_ISA_wot_1[i], π_ISA_wot_2[i]], extrapolation_bc = Line())
        π_ISA_wot_data[i,:] = interp_π_ISA_wot_1D.(δ_range_π)
    end

    interp_π_ISA_wot = LinearInterpolation((n_range_π,δ_range_π), π_ISA_wot_data, extrapolation_bc = ((Flat(), Flat()), (Flat(), Line())))

end

interp_π_ISA_wot = build_interp_π_wot()

@show interp_π_ISA_wot(2300/ω_rated, δ_from_h(24000*.3048))*P_rated
@show interp_π_ISA_wot(1900/ω_rated, δ_from_h(12000*.3048))*P_rated

let
# n_plot = [0.15ω_rated, 0.25ω_rated, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700]/ω_rated
n_plot = range(0.15, 1.5, length = 21)
δ_plot = range(1, δ_from_h(40000*0.3048), length = 10)
π_wot_plot = [interp_π_ISA_wot(n,p) for (n,p) in Iterators.product(n_plot, δ_plot)]
# plot(δ_plot, π_wot_plot')
plot(n_plot, π_wot_plot)
end


UndefVarError: UndefVarError: interp_μ_wot not defined

In [8]:
# function compute_Pbar_ISA(thr, nbar, δ)

#     #compute the wide-open throttle MAP for the given RPMs and altitude
#     @show Mbar_wot = interp_Mbar_wot(nbar, δ)
#     idle_MAP_ratio = 0.4
#     #this can be tuned so that the engine idles at appropriate RPMs with the
#     #chosen propeller
#     @show Mbar = Mbar_wot * (idle_MAP_ratio + thr * (1 - idle_MAP_ratio))

#     #δ at which our Mbar would be Mbar_wot
#     @show δ_wot = interp_δ_wot(nbar, Mbar)

#     Pbar_ISA_std = interp_Pbar_ISA_std(nbar, Mbar)
#     Pbar_ISA_wot = interp_Pbar_ISA_wot(nbar, δ_wot)
#     @show P_ISA_std = Pbar_ISA_std * P_rated
#     @show P_ISA_wot = Pbar_ISA_wot * P_rated
    
#     #when p_wot is close to p_std with MAP = MAP_wot (thr = 1),
#     #p_wot(MAP_wot(p_std)) = p_std and P̃_wot = P̃_std. we need to avoid the
#     #division by zero
#     @show abs(δ_wot - 1)
#     if abs(δ_wot - 1) < 1e-3
#         @show "Hi"
#         Pbar_ISA = Pbar_ISA_std
#     else
#         Pbar_ISA = Pbar_ISA_std + (Pbar_ISA_wot - Pbar_ISA_std) / (δ_wot - 1) * (δ - 1)
#         @show Pbar_ISA_std
#     end

#     @show P_ISA = Pbar_ISA * P_rated
#     @show MAP = Mbar * MAP_rated
#     @show δ = Mbar * MAP_rated

#     return max(0, Pbar_ISA)

# end


In [9]:
thr_test = 1
nbar_test = 2700/n_rated
δ_test = compute_δ_h(30000 * 0.3048)
compute_Pbar_ISA(thr_test, nbar_test, δ_test) * P_rated

UndefVarError: UndefVarError: n_rated not defined